In [155]:
''' 

in Json speichern


Ausgangslage: Programmierung kopiert von https://github.com/deutsche-nationalbibliothek/dnblab/blob/main/DNB_digitale_Inhaltsverzeichnisse_Tutorial.ipynb


Die Abhängigkeiten der Programmierung sind angepasst, so dass sie ins VOEBB-Forum geladen werden können.

Ziel der Programmierung ist es:
ich extrahiere die ISBNs eines Metadatesatzes als Array, gebe die ISBNs nacheinander in die Abfrage ein 
und erhalte am Schluss den Metadatensatz angereichert mit dem Inhaltsverzeichnis in der Spalte „Inhaltsverzeichnis“.

Vom Exception Handling werden folgende Dinge abgedeckt:

es gibt kein Medium mit der ISBN auf den DNB-Seiten (ausländisches Medium)
es gibt kein Inhaltsverzeichnis zu dem Medium auf den DNB-Seiten
es gibt Verbindungsprobleme bei der Stapelverarbeitung. In dem Fall versucht er insgesamt 5 Mal in Abstand von 8s.


Die Fuktionen dnb_sru und parse_record sind unverändert aus der DNB-Programmierung übernommen. 

ToDo:
Copywarning am Ende adressieren
Die Speicherung als Excel funktioniert nicht. Speicherdatenformat überdenken - wie geht der Datenflow weiter?


'''


' \n\nin Json speichern\n\n\nAusgangslage: Programmierung kopiert von https://github.com/deutsche-nationalbibliothek/dnblab/blob/main/DNB_digitale_Inhaltsverzeichnisse_Tutorial.ipynb\n\n\nDie Abhängigkeiten der Programmierung sind angepasst, so dass sie ins VOEBB-Forum geladen werden können.\n\nZiel der Programmierung ist es:\nich extrahiere die ISBNs eines Metadatesatzes als Array, gebe die ISBNs nacheinander in die Abfrage ein \nund erhalte am Schluss den Metadatensatz angereichert mit dem Inhaltsverzeichnis in der Spalte „Inhaltsverzeichnis“.\n\nVom Exception Handling werden folgende Dinge abgedeckt:\n\nes gibt kein Medium mit der ISBN auf den DNB-Seiten (ausländisches Medium)\nes gibt kein Inhaltsverzeichnis zu dem Medium auf den DNB-Seiten\nes gibt Verbindungsprobleme bei der Stapelverarbeitung. In dem Fall versucht er insgesamt 5 Mal in Abstand von 8s.\n\n\nDie Fuktionen dnb_sru und parse_record sind unverändert aus der DNB-Programmierung übernommen. \n\nToDo:\nCopywarning am End

In [156]:
# Stand 27. Okt. 2025

In [157]:


import re
import requests
import unicodedata
from lxml import etree
import pandas as pd
from bs4 import BeautifulSoup as soup
from collections import Counter
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
from datetime import datetime


In [158]:
#laden der mit den inhaltsverzeichnissen anzureichernden Datei
#

df = pd.read_excel('20251010_dummy_titelliste_4.xlsx')


In [159]:
# hinzufügen der Splate "inhaltsverzeichnis"

df['inhaltsverzeichnis']=pd.Series(dtype="str")
df['dcc']=pd.Series(dtype="str")

In [160]:
# array mit ISBN-Werten erstellen
isbn=df['ISBN']
for i in isbn:
    print(i)

0-415-17473-2
0-571-52576-8
0-634-00901-X
0-7119-2071-0
0-7670-5811-9
3-211-83467-2
3-445-04765-0
3-446-20529-2
3-453-07338-X
3-464-60186-2
3-478-86024-5
3-496-02668-5
3-518-02780-8
3-518-22256-2
3-518-29138-6
3-518-38447-3
3-518-41383-X
3-608-91026-3
3-7466-6065-3
3-86099-317-8
3-88377-102-3
3-89620-150-6
3-95983-509-4
893-2000-10692-0
978-0-00-855628-0
978-0-00-859793-1
978-0-06-293691-2
978-0-307-94982-0
978-0-367-50040-5
978-0-385-35398-4
978-0-385-54861-8
978-0-86356-917-3
978-1-03-235538-2
978-1-03-247492-2
978-1-107-68183-5
978-1-108-79492-3
978-1-250-32675-1
978-1-316-50077-4
978-1-394-15345-9
978-1-398-70818-1
978-1-399-71148-7
978-1-3994-1242-1
978-1-4063-9922-6
978-1-4696-2660-4
978-1-4742-4237-0
978-1-5400-5840-9
978-1-63681-093-5
978-1-64129-464-5
978-1-6626-6514-1
978-1-68152-856-4
978-1-70513-144-2
978-1-70516-866-0
978-1-70516-868-4
978-1-70518-758-6
978-1-70519-537-6
978-1-70519-600-7
978-1-78658-264-5
978-1-78873-492-9
978-1-84328-054-5
978-1-84861-856-5
978-2-02-1525

In [161]:
# Funktion kopiert aus einem DNB Workshop im Netz:
#https://github.com/deutsche-nationalbibliothek/dnblab/blob/main/DNB_digitale_Inhaltsverzeichnisse_Tutorial.ipynb
# scraped den zu der isbn gehörenden kompletten datensatz von der internetseite

def dnb_sru(query):
    
    base_url = "https://services.dnb.de/sru/dnb"
    params = {'recordSchema' : 'MARC21-xml',
          'operation': 'searchRetrieve',
          'version': '1.1',
          'maximumRecords': '10',
          'query': query
         }
    r = requests.get(base_url, params=params)
    xml = soup(r.content, features="xml")
    records = xml.find_all('record', {'type':'Bibliographic'})
    
    if len(records) < 10:
        
        return records
    
    else:
        
        num_results = 10
        i = 11
        while num_results == 10:
            
            params.update({'startRecord': i})
            r = requests.get(base_url, params=params)
            xml = soup(r.content, features="xml")
            new_records = xml.find_all('record', {'type':'Bibliographic'})
            records+=new_records
            i+=10
            num_results = len(new_records)
            
        return records

In [162]:
# Funktion kopiert aus einem DNB Workshop im Netz:
#https://github.com/deutsche-nationalbibliothek/dnblab/blob/main/DNB_digitale_Inhaltsverzeichnisse_Tutorial.ipynb
# scraped den link zum inhaltsverzeichnis aus dem record datensatz
# hier kannst du dann auch andere Marc-Felder wie SChlagwort adressieren

def parse_record(record):
    
    ns = {"marc":"http://www.loc.gov/MARC21/slim"}
    xml = etree.fromstring(unicodedata.normalize("NFC", str(record)))
    
    
    descriptions = xml.xpath("marc:datafield[@tag = '856']/marc:subfield[@code = '3']", namespaces=ns)
    links = xml.xpath("marc:datafield[@tag = '856']/marc:subfield[@code = 'u']", namespaces=ns)
    
    try:
        description1 = descriptions[0].text
    except:
        description1 = "unknown"
        
    try: 
        description2 = descriptions[1].text
    except:
        description2 = "unknown"
        
        
    if description1 == "Inhaltsverzeichnis": 
        link = links[0].text
    elif description2 == "Inhaltsverzeichnis": 
        link = links[1].text
    else:
        link = "unknown"

    
        
        
    meta_dict = {"link":link + '/text'}
    
    return meta_dict

In [163]:
# initiert die Variable mit der der Index des Dataframes in der untenstehenden for-Schleife 
#trotz exception handling adressiert werden kann
t=-1 

# falls die Verbindung zur Webseite abgelehnt wird, ist das eine Funktion die die 
#Wiederholung der Abfrage organisiert

session = requests.Session()

retries = Retry(
    total=5,                    # total attempts
    backoff_factor=8,           # wait 1, 2, 4, 8, ... seconds between retries
    status_forcelist=[500, 502, 503, 504],  # retry on these HTTP codes
    allowed_methods=["GET", "POST"]
)

In [164]:
for i in isbn:
    records = dnb_sru(i)
 
    #Inhaltsverzeichnis scrapen
    inhaltsverzeichnis = [parse_record(record) for record in records]
    x = pd.DataFrame(inhaltsverzeichnis)
    print(x)
    # "t" nimmt den Index des Dataframes in den das Inhaltsverzeichnis geladen werden soll
    # mit in die for-Schleife (s.u.)
    t += 1

    # wenn der Titel kein Inhaltsverzeichnis hat
    if "unknown/text" in str(x):
        continue

    # wenn der titel nicht in der dnb vorkommt (ausländischer titel)
    if inhaltsverzeichnis == []:
        continue
    
    # url speichert den Link zum Inhaltsverzeichnis
    url=x['link'].iloc[0]
    
    # Filename speichert das Inhaltsverzeichnis unter isbn +'.txt'
    filename = str(i) + ".txt"
    
    # folgende Angaben organisieren die wiederholte Anfrage an die DNB-Seite, wenn die Verbindung 
    # abgelehnt wurde und den Download der Datei
    adapter = HTTPAdapter(max_retries=retries)
    session.mount("http://", adapter)
    session.mount("https://", adapter)
    
    
    try:
        r = requests.get(url, timeout=15)
        r.raise_for_status()
        # Inhalt als Text (Decoder kümmert sich um UTF-8)
        text = r.text


        with open(filename, "wb") as f:
            f.write(r.content)
            print("Download successful:", filename)

        
        # Zeilenumbrüche im Inhaltsverzeichnis entfernen
        text_cleaned = text.replace("\r", "").replace("\n", "")
        
        # Inhaltsverzeichnis im Dataframe unter der Spalte "Inhaltsverzeichnis" speichern
        with open(filename, "r", encoding="latin-1") as f:
            text = f.read().replace("\r", " ").replace("\n", " ")
            df.loc[t,'inhaltsverzeichnis'] = text   # speichert text in richigen Reihe und Spalte des Dataframe
            print("Download successful:", filename)
    except requests.exceptions.RequestException as e:
        print("Download failed:", e)
    
    
    # speichert die inhaltsverzeichnisse in die falsche Zeile ????

Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
                                  link
0  https://d-nb.info/961917180/04/text
Download successful: 3-211-83467-2.txt
Download successful: 3-211-83467-2.txt
                                  link
0  https://d-nb.info/910514186/04/text
Download successful: 3-445-04765-0.txt
Download successful: 3-445-04765-0.txt
           link
0  unknown/text
                                  link
0  https://d-nb.info/940754797/04/text
Download successful: 3-453-07338-X.txt
Download successful: 3-453-07338-X.txt
                                  link
0  https://d-nb.info/973062738/04/text
Download successful: 3-464-60186-2.txt
Download successful: 3-464-60186-2.txt
                                  link
0  https://d-nb.info/966102479/04/text
Download successful: 3-478-86024-5.txt
Download successful: 3-478-86024-5.tx

                                                link
0  https://www.degruyter.com/isbn/9783110683714/text
Download failed: 404 Client Error: Not Found for url: https://www.degruyterbrill.com:443/isbn/9783110683714/text
                                                link
0  https://www.degruyter.com/isbn/9783110697346/text
Download failed: 404 Client Error: Not Found for url: https://www.degruyterbrill.com:443/isbn/9783110697346/text
                                                link
0  https://www.degruyter.com/isbn/9783110713916/text
Download failed: 404 Client Error: Not Found for url: https://www.degruyterbrill.com:443/isbn/9783110713916/text
                                                link
0  https://www.degruyter.com/isbn/9783110714661/text
Download failed: 404 Client Error: Not Found for url: https://www.degruyterbrill.com:443/isbn/9783110714661/text
                                                link
0  https://www.degruyter.com/isbn/9783110782356/text
Download failed: 4

Download successful: 978-3-351-04183-0.txt
Download successful: 978-3-351-04183-0.txt
                                   link
0  https://d-nb.info/128818123X/04/text
Download successful: 978-3-374-07453-2.txt
Download successful: 978-3-374-07453-2.txt
                                   link
0  https://d-nb.info/1322458561/04/text
1  https://d-nb.info/1244401242/04/text
2  https://d-nb.info/1038502306/04/text
3  https://d-nb.info/1013812166/04/text
4   https://d-nb.info/965753670/04/text
5   https://d-nb.info/953531058/04/text
6   https://d-nb.info/947355308/04/text
7   https://d-nb.info/947355235/04/text
8   https://d-nb.info/945151454/04/text
Download successful: 978-3-403-02645-7.txt
Download successful: 978-3-403-02645-7.txt
                                   link
0  https://d-nb.info/1160201382/04/text
Download successful: 978-3-403-08104-3.txt
Download successful: 978-3-403-08104-3.txt
                                   link
0  https://d-nb.info/1291592547/04/text
Download success

Download successful: 978-3-440-17351-0.txt
Download successful: 978-3-440-17351-0.txt
                                   link
0  https://d-nb.info/1283637774/04/text
Download successful: 978-3-440-17420-3.txt
Download successful: 978-3-440-17420-3.txt
                                   link
0  https://d-nb.info/1283634171/04/text
Download successful: 978-3-440-17681-8.txt
Download successful: 978-3-440-17681-8.txt
           link
0  unknown/text
           link
0  unknown/text
           link
0  unknown/text
           link
0  unknown/text
           link
0  unknown/text
           link
0  unknown/text
                                   link
0  https://d-nb.info/1034157981/04/text
Download successful: 978-3-442-74652-1.txt
Download successful: 978-3-442-74652-1.txt
                                   link
0  https://d-nb.info/122071769X/04/text
Download successful: 978-3-442-75924-8.txt
Download successful: 978-3-442-75924-8.txt
           link
0  unknown/text
1  unknown/text
          

           link
0  unknown/text
           link
0  unknown/text
           link
0  unknown/text
                                   link
0  https://d-nb.info/1289523304/04/text
Download successful: 978-3-494-01967-3.txt
Download successful: 978-3-494-01967-3.txt
                                   link
0  https://d-nb.info/1200131339/04/text
Download successful: 978-3-495-49106-5.txt
Download successful: 978-3-495-49106-5.txt
                                   link
0  https://d-nb.info/1293527548/04/text
Download successful: 978-3-495-99520-4.txt
Download successful: 978-3-495-99520-4.txt
                                   link
0  https://d-nb.info/1283688387/04/text
Download successful: 978-3-495-99603-4.txt
Download successful: 978-3-495-99603-4.txt
                                   link
0  https://d-nb.info/1277296308/04/text
Download successful: 978-3-495-99780-2.txt
Download successful: 978-3-495-99780-2.txt
                                   link
0  https://d-nb.info/1277660506/04

...

                                   link
0  https://d-nb.info/1256447358/04/text
Download successful: 978-3-8353-5300-8.txt
Download successful: 978-3-8353-5300-8.txt
                                   link
0  https://d-nb.info/1273626419/04/text
Download successful: 978-3-8353-5375-6.txt
Download successful: 978-3-8353-5375-6.txt
                                   link
0  https://d-nb.info/1283338521/04/text
Download successful: 978-3-8362-9554-3.txt
Download successful: 978-3-8362-9554-3.txt
                                   link
0  https://d-nb.info/1284099458/04/text
Download successful: 978-3-8369-2198-5.txt
Download successful: 978-3-8369-2198-5.txt
                                   link
0  https://d-nb.info/118198193X/04/text
Download successful: 978-3-8369-5649-9.txt
Download successful: 978-3-8369-5649-9.txt
           link
0  unknown/text
                                   link
0                          unknown/text
1  https://d-nb.info/1034225448/04/text
                  

                                   link
0  https://d-nb.info/1050665740/04/text
Download successful: 978-3-86351-086-2.txt
Download successful: 978-3-86351-086-2.txt
                                   link
0  https://d-nb.info/1291828923/04/text
Download successful: 978-3-86371-367-6.txt
Download successful: 978-3-86371-367-6.txt
                                   link
0  https://d-nb.info/1309263582/04/text
Download successful: 978-3-86371-399-7.txt
Download successful: 978-3-86371-399-7.txt
                                   link
0  https://d-nb.info/1275631576/04/text
Download successful: 978-3-86408-296-2.txt
Download successful: 978-3-86408-296-2.txt
                                   link
0  https://d-nb.info/1275630278/04/text
Download successful: 978-3-86408-298-6.txt
Download successful: 978-3-86408-298-6.txt
                                   link
0  https://d-nb.info/1293186325/04/text
Download successful: 978-3-86442-419-9.txt
Download successful: 978-3-86442-419-9.txt
    

                                   link
0  https://d-nb.info/1293524298/04/text
Download successful: 978-3-910851-01-6.txt
Download successful: 978-3-910851-01-6.txt
                                   link
0  https://d-nb.info/1028936281/04/text
Download successful: 978-3-927372-78-8.txt
Download successful: 978-3-927372-78-8.txt
           link
0  unknown/text
           link
0  unknown/text
                                   link
0  https://d-nb.info/1002361184/04/text
Download successful: 978-3-932574-16-0.txt
Download successful: 978-3-932574-16-0.txt
           link
0  unknown/text
           link
0  unknown/text
                                  link
0  https://d-nb.info/988957701/04/text
1                         unknown/text
                                   link
0  https://d-nb.info/1018210105/04/text
Download successful: 978-3-937445-49-6.txt
Download successful: 978-3-937445-49-6.txt
                                   link
0  https://d-nb.info/1064604978/04/text
Download su

           link
0  unknown/text
           link
0  unknown/text
                                   link
0  https://d-nb.info/1209899655/04/text
Download successful: 978-3-95972-375-6.txt
Download successful: 978-3-95972-375-6.txt
                                   link
0  https://d-nb.info/1248828585/04/text
Download successful: 978-3-95982-508-5.txt
Download successful: 978-3-95982-508-5.txt
                                   link
0  https://d-nb.info/1248829573/04/text
Download successful: 978-3-95982-511-5.txt
Download successful: 978-3-95982-511-5.txt
                                   link
0  https://d-nb.info/1292391766/04/text
Download successful: 978-3-95983-642-5.txt
Download successful: 978-3-95983-642-5.txt
                                   link
0  https://d-nb.info/1275236510/04/text
Download successful: 978-3-95985-666-9.txt
Download successful: 978-3-95985-666-9.txt
           link
0  unknown/text
                                   link
0  https://d-nb.info/1286756367/04

                                   link
0  https://d-nb.info/1269426621/04/text
Download successful: 978-3-99069-854-9.txt
Download successful: 978-3-99069-854-9.txt
           link
0  unknown/text
           link
0  unknown/text
                                   link
0  https://d-nb.info/1295219832/04/text
Download successful: 978-3-99144-004-8.txt
Download successful: 978-3-99144-004-8.txt
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
           link
0  unknown/text
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
Em

In [165]:
df['inhaltsverzeichnis']

0      NaN
1      NaN
2      NaN
3      NaN
4      NaN
      ... 
876    NaN
877    NaN
878    NaN
879    NaN
880    NaN
Name: inhaltsverzeichnis, Length: 881, dtype: object

In [166]:
# im folgenden den dataframe als json speichern

df.to_json("ki_metadaten.json", orient="records", indent=2, force_ascii=False)




In [ ]:
# kontrolle
'''

978-3-0358-0042-5    kein INhaltsverzeichnis / korrekt
0-415-17473-2        kein Treffer  / korrekt
978-3-518-22511-0   hat zwei inhaltsverzeichnisse keins wird geladen / Exception, meines Erachtens korrekt
978-3-942449-36-6   hat inhaltsverzeichnis / korrekt
978-3-8376-2446-5   isbn doppelt in dnb / kein inhaltsverzeichnis / Exception, meines Erachtens korrekt
978-3-7416-2280-9   hat inhaltsverzeichnis / korrekt
978-3-7416-1018-9   kein inhaltsverzeichnis / korrekt
978-3-7416-0481-2   inhaltsverzeichni / korrekt
978-3-86201-588-7   kein inhaltsverzeichnis / korrekt
978-3-86201-326-5   kein inhaltsverzeichnis / korrekt
978-3-7416-2003-4   inhaltsverzeichnis / korrekt
978-3-608-98658-7   kein inhaltsverzeichnis / korrekt
978-3-946807-54-4   kein inhaltsverzeichnis / korrekt
978-3-95972-375-6   inhaltsverzeichnis / korrekt
978-3-7074-5294-5   inhaltsverzeichnis / korrekt
978-3-98921-091-2   kein inhaltsverzeichnis / korrekt


# bei der Datenbereinigung 200 von 1.000 Titel ohne ISBN


'''




In [22]:
# dataframe als excel speichern
# funktioniert nicht - excel scheint auch nicht das richige Speicherformat.
# Wie geht der Datenflow weiter?


# aktuelles Datum im Format JJJJ-MM-TT
#heute = datetime.now().strftime("%Y-%m-%d")




#df.to_excel(heute+'_inhaltverzeichnisse_anreichern.xlsx')

In [ ]:
'''
# version in dem die Zeilenbrüche imt Inhaltsverzeichnis beibehalten werden:

for i in isbn:
    records = dnb_sru(i)
    # schlagwort scrapen
#    inhaltsverzeichnis = [parse_record_schlag(record) for record in records]
#    x = pd.DataFrame(inhaltsverzeichnis)    

    
    #INhaltsverzeichnis scrapen
    inhaltsverzeichnis = [parse_record(record) for record in records]
    x = pd.DataFrame(inhaltsverzeichnis)

    # "t" nimmt den Index des Dataframes in den das Inhaltsverzeichnis geladen werden soll
    # mit in die for-Schleife (s.u.)
    t += 1

    # wenn der Titel kein Inhaltsverzeichnis hat
    if "unknown/text" in str(x):
        continue

    # wenn der titel nicht in der dnb vorkommt (ausländischer titel)
    if inhaltsverzeichnis == []:
        continue
    
    # url speichert den Link zum Inhaltsverzeichnis
    url=x['link'].iloc[0]
    
    # Filename speichert das Inhaltsverzeichnis unter isbn +'.txt'
    filename = str(i) + ".txt"
    
    # folgende Angaben organisieren die wiederholte Anfrage an die DNB-Seite, wenn die Verbindung 
    # abgelehnt wurde und den Download der Datei
    adapter = HTTPAdapter(max_retries=retries)
    session.mount("http://", adapter)
    session.mount("https://", adapter)
    
    
    try:
        r = requests.get(url, timeout=15)
        r.raise_for_status()
        with open(filename, "wb") as f:
            f.write(r.content)
            print("Download successful:", filename)
    except requests.exceptions.RequestException as e:
        print("Download failed:", e)
    
    # lädt die Donwload-Datei und speichert sie in den Dataframe mit den Metadaten.
    z=pd.read_fwf(filename)   # gespeicherte datei öffnen und in der variablen "z"speichern
    df['inhaltsverzeichnis'].iloc[t]= z   # z in richigen Reihe und Spalte des Dataframe speichern
    






'''

In [ ]:
# Version mit downöoad ders inhatlsverzeichnisses als Block
for i in isbn:
    records = dnb_sru(i)
    # schlagwort scrapen
#    inhaltsverzeichnis = [parse_record_schlag(record) for record in records]
#    x = pd.DataFrame(inhaltsverzeichnis)    
  
    #Inhaltsverzeichnis scrapen
    inhaltsverzeichnis = [parse_record(record) for record in records]
    x = pd.DataFrame(inhaltsverzeichnis)

    # "t" nimmt den Index des Dataframes in den das Inhaltsverzeichnis geladen werden soll
    # mit in die for-Schleife (s.u.)
    t += 1

    # wenn der Titel kein Inhaltsverzeichnis hat
    if "unknown/text" in str(x):
        continue

    # wenn der titel nicht in der dnb vorkommt (ausländischer titel)
    if inhaltsverzeichnis == []:
        continue
    
    # url speichert den Link zum Inhaltsverzeichnis
    url=x['link'].iloc[0]
    
    # Filename speichert das Inhaltsverzeichnis unter isbn +'.txt'
    filename = str(i) + ".txt"
    
    # folgende Angaben organisieren die wiederholte Anfrage an die DNB-Seite, wenn die Verbindung 
    # abgelehnt wurde und den Download der Datei
    adapter = HTTPAdapter(max_retries=retries)
    session.mount("http://", adapter)
    session.mount("https://", adapter)
    
    
    try:
        r = requests.get(url, timeout=15)
        r.raise_for_status()
        # Inhalt als Text (Decoder kümmert sich um UTF-8)
        text = r.text

        # Zeilenumbrüche entfernen
        text_cleaned = text.replace("\r", "").replace("\n", "")

        # Speichern als Text
        with open(filename, "w", encoding="utf-8") as f:
            f.write(text_cleaned)
            
            print("Download successful:", filename)
    except requests.exceptions.RequestException as e:
        print("Download failed:", e)
    
    # lädt die Donwload-Datei und speichert sie in den Dataframe mit den Metadaten.
    z=pd.read_fwf(filename)   # gespeicherte datei öffnen und in der variablen "z"speichern
    df['inhaltsverzeichnis'].iloc[t]= z   # z in richigen Reihe und Spalte des Dataframe speichern
    
    
    
    
    
    # Pfad zur Textdatei
filename = "mein_text.txt"

# Datei lesen und Zeilenumbrüche entfernen
with open(filename, "r", encoding="utf-8") as f:
    text = f.read().replace("\r", " ").replace("\n", " ")

# z. B. in Zeile mit Index 0 schreiben:
df.loc[0, "inhalt"] = text

print(df)
    
  

In [169]:
df['inhaltsverzeichnis'].count()

519